In [50]:
import pandas as pd
import numpy as np
import os
import re
import ast

In [51]:
data_dir = '/Users/xiaoqianxiao/UKB/data'
participantsInfo_file = 'participants.csv'
participantsInfo_file_path = os.path.join(data_dir,participantsInfo_file)
participantsInfo = pd.read_csv(participantsInfo_file_path)

In [52]:
#demographic factors' field ids, including: eid, sex, age at the first scan, IQ and EA (5)
demographic_fields = ['31','21003_2','20016_2','6138_2']
current_status_fields = ['2050_2','2060_2','2070_2']
# Self_Reported_Mental_Health (1)
Self_Reported_Mental_Health_fields = ['29000','20002_i2','21062']
# Ever_Diagnosed_Mental_Health_Problem (1)
Ever_Diagnosed_Mental_Health_Problem_fields = ['20544']
# Self reported history depression: CIDI (13)
history_depression_fields = ['20436','20439','20440','20446','20441','20449','20536','20532','20435','20450','20437']
# Self reported history anxiety: CIDI (18)
history_anxiety_fields = ['20421','20420','20538','20425','20542','20543','20540','20541','20539','20537','20418','20426','20423','20429','20419','20422','20417','20427']
# PHQ (9)
PHQ_fields = ['20514','20510','20517','20519','20511','20507','20508','20518','20513']
# GAD7 (7)
GAD7_fields = ['20506','20509','20520','20515','20516','20505','20512']
# hospital data: ICD10 and ICD9 (2)
hospital_data_fields = ['41270','41271']
# fMRI data
fMRI_fields = ['31016','31018','31019','31015','31014']

In [53]:
def fields_for_id_x(field_id):
    field_id = str(field_id)
    field_items = re.split(r'[,\s_]+', field_id)
    if len(field_items) == 1:
        fields = 'p{}'.format(field_items[0])
    elif len(field_items) == 2:
        fields = 'p{}_i{}'.format(field_items[0], field_items[1])
    return fields

def extract_strings(string_patten, strings_list):
    # Filter strings that start with "F4"
    #return [s for s in strings_list if s.startswith(string_patten)]
    return [s for s in strings_list if any(pattern in s for pattern in string_patten)]

def safe_literal_eval(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except (ValueError, SyntaxError):
        return []

In [54]:
# transfer EA
dl = []
for i in range(participantsInfo.shape[0]):
    # Prefer not to answer
    if participantsInfo['p6138_i2'].iloc[i] == -3:
        dl.append(participantsInfo.index[i])
    if participantsInfo['p6138_i2'].iloc[i] == 1:
        participantsInfo['p6138_i2'].iloc[i] = 20
    if participantsInfo['p6138_i2'].iloc[i] == 2:
        participantsInfo['p6138_i2'].iloc[i] = 13
    if participantsInfo['p6138_i2'].iloc[i] == 3:
        participantsInfo['p6138_i2'].iloc[i] = 10
    if participantsInfo['p6138_i2'].iloc[i] == 4:
        participantsInfo['p6138_i2'].iloc[i] = 10
    if participantsInfo['p6138_i2'].iloc[i] == 5:
        participantsInfo['p6138_i2'].iloc[i] = 19 
    if participantsInfo['p6138_i2'].iloc[i] == 6:
        participantsInfo['p6138_i2'].iloc[i] = 15
    # None of the above
    if participantsInfo['p6138_i2'].iloc[i] == -7:
        dl.append(participantsInfo.index[i])   
participantsInfo.drop(dl,inplace=True)

In [55]:
# get subset with participants have fMRI data series:
participants_withfMRI = participantsInfo.dropna(subset=['p31016_i2'])
print("number of participants with fMRI time series: {}".format(participants_withfMRI.shape[0]))
df_fMRI = pd.DataFrame(participants_withfMRI['eid'])

number of participants with fMRI time series: 40396


In [56]:
l1 = []
l2 = []
for i in range(participants_withfMRI.shape[0]):
    #depression
    if participants_withfMRI['p2050_i2'].iloc[i] >= 3 or participants_withfMRI['p2060_i2'].iloc[i] >= 3:
        l1.append(participants_withfMRI['eid'].iloc[i])
    #anxiety
    if participants_withfMRI['p2070_i2'].iloc[i] >= 3:
        l2.append(participants_withfMRI['eid'].iloc[i])

In [57]:
#CIDI
history_anxiety_fields = ['20421','20420','20538','20425','20542','20543','20540','20541','20539','20537','20418','20426','20423','20429','20419','20422','20417','20427']
df_fMRI['history_anxiety_num_criterias'] = None
df_fMRI['history_anxiety_num_symptoms'] = None
history_anxiety_subjs = []
for i in range(participants_withfMRI.shape[0]):
    num_criterias = 0
    if participants_withfMRI['p20421'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if participants_withfMRI['p20420'].iloc[i] >= 6 or participants_withfMRI['p20420'].iloc[i] == -999:
        num_criterias = num_criterias + 1
    if participants_withfMRI['p20538'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if participants_withfMRI['p20425'].iloc[i] == 1 or participants_withfMRI['p20542'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if participants_withfMRI['p20543'].iloc[i] == 2 or participants_withfMRI['p20540'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if participants_withfMRI['p20541'].iloc[i] == 1 or participants_withfMRI['p20539'].iloc[i] == 3 or participants_withfMRI['p20537'].iloc[i] == 3:
        num_criterias = num_criterias + 1
    if participants_withfMRI['p20418'].iloc[i] >= 2:
        num_criterias = num_criterias + 1
    
    # 3 somatic symptoms out of
    num_symptoms = 0
    if participants_withfMRI['p20426'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if participants_withfMRI['p20423'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if participants_withfMRI['p20429'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if participants_withfMRI['p20419'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if participants_withfMRI['p20422'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if participants_withfMRI['p20417'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if participants_withfMRI['p20427'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
        
    if num_symptoms >= 3:
        num_criterias = num_criterias + 1
    
    if num_criterias == 8:
        history_anxiety_subjs.append(participants_withfMRI['eid'].iloc[i])
        
    df_fMRI.iloc[i, df_fMRI.columns.get_loc('history_anxiety_num_criterias')] = num_criterias
    df_fMRI.iloc[i, df_fMRI.columns.get_loc('history_anxiety_num_symptoms')] = num_symptoms
df_fMRI['history_anxiety'] = df_fMRI['history_anxiety_num_criterias'].apply(lambda x: (x == 8)).replace((1,0),('True','False'))
print("number of history anxiety: ", df_fMRI.loc[df_fMRI['history_anxiety']==True, 'history_anxiety'].shape[0])

number of history anxiety:  1909


In [130]:
# Current anxiety
GAD7_fields_name = [fields_for_id_x(field) for field in GAD7_fields]
df_fMRI['GAD7_score'] = participants_withfMRI[GAD7_fields_name].sum(axis = 1)
df_fMRI['Current_Anxiety'] = False
df_fMRI.loc[df_fMRI['GAD7_score'] >= 10,'Current_Anxiety'] = True
df_fMRI.loc[df_fMRI['GAD7_score'] < 5,'Current_no_Anxiety'] = True
print('number of Diagnosed in Current_Anxiety:', df_fMRI['Current_Anxiety'].sum())

number of Diagnosed in Current_Anxiety: 7799


In [115]:
# hospital data-linkage: 
# Data-Field 41270 
df_fMRI['ICD10'] = participants_withfMRI['p41270'].apply(lambda x: extract_strings(['F40', 'F41', 'F42'], safe_literal_eval(x)) if isinstance(safe_literal_eval(x), list) else [])
df_fMRI['Diagnosed_ICD10'] = False
df_fMRI.loc[df_fMRI['ICD10'].apply(lambda x: x != []), 'Diagnosed_ICD10'] = True
print('number of Diagnosed in ICD10:', df_fMRI['Diagnosed_ICD10'].sum())
# Data-Field 41271 
df_fMRI['ICD9'] = participants_withfMRI['p41271'].apply(lambda x: extract_strings('300', safe_literal_eval(x)) if isinstance(safe_literal_eval(x), list) else [])
df_fMRI['Diagnosed_ICD9'] = False
df_fMRI.loc[df_fMRI['ICD9'].apply(lambda x: x != []), 'Diagnosed_ICD9'] = True
print('number of Diagnosed in ICD9:', df_fMRI['Diagnosed_ICD9'].sum())

number of Diagnosed in ICD10: 1426
number of Diagnosed in ICD9: 856


In [117]:
# Generate number of subjects in different subcategories
# hospital data-linkage: 
# for OCD
# Data-Field 41270 
df_fMRI['ICD10_OCD'] = participants_withfMRI['p41270'].apply(lambda x: extract_strings(['F42'], safe_literal_eval(x)) if isinstance(safe_literal_eval(x), list) else [])
df_fMRI['Diagnosed_ICD10_OCD'] = False
df_fMRI.loc[df_fMRI['ICD10_OCD'].apply(lambda x: x != []), 'Diagnosed_ICD10_OCD'] = True
print('number of Diagnosed as OCD in ICD10:', df_fMRI['Diagnosed_ICD10_OCD'].sum())

number of Diagnosed as OCD in ICD10: 16


Series([], Name: p41270, dtype: object)

In [76]:
# Self reported
# 1. self-reporting a lifetime professional diagnosis of one of the core five anxiety disorders [Data-Field 29000]
Self_Reported_MedicalCondition = participants_withfMRI.filter(like='29000')
anxiety_code = list(range(10,16))

for i in range(len(anxiety_code)):
    code = anxiety_code[i]
    #print(code)
    added_item = 'Self_Reported_diagnosis'+ str(code)
    df_fMRI[added_item] = Self_Reported_MedicalCondition['p29000'].apply(lambda row: (code in safe_literal_eval(row)) if isinstance(safe_literal_eval(row), list) else [])

columns_Self_Reported_diagnosis = df_fMRI.filter(like='Self_Reported_diagnosis')
df_fMRI['Self_Reported_diagnosis'] = columns_Self_Reported_diagnosis.apply(lambda row: (row == True).any(), axis=1).replace((1,0),('True','False'))
print("number of self-reporting of diagnosis: ", df_fMRI.loc[df_fMRI['Self_Reported_diagnosis']==True, 'Self_Reported_diagnosis'].shape[0])

number of self-reporting of diagnosis:  2025


In [77]:
# 2. Self-reporting of medical condition:Non-cancer illness code, self-reported[Data-Field 20002]
Self_Reported_MedicalCondition = participants_withfMRI.filter(like='20002_i2')
anxiety_code = [1615,1287]
for i in range(len(anxiety_code)):
    code = anxiety_code[i]
    #print(code)
    added_item = 'Self_Reported_MedicalCondition'+ str(code)
    df_fMRI[added_item] = Self_Reported_MedicalCondition.apply(lambda row: (row == code).any(), axis=1).replace((1,0),('True','False'))
    
columns_Self_Reported_MedicalCondition = df_fMRI.filter(like='Self_Reported_MedicalCondition')
df_fMRI['Self_Reported_MedicalCondition'] = columns_Self_Reported_MedicalCondition.apply(lambda row: (row == True).any(), axis=1).replace((1,0),('True','False'))
print("number of self-reporting of medical conditions: ", df_fMRI.loc[df_fMRI['Self_Reported_MedicalCondition']==True, 'Self_Reported_MedicalCondition'].shape[0])

number of self-reporting of medical conditions:  1096


In [78]:
# 3. Self-reporting of Ever been offered/sought treatment for anxiety [Data-Field 21062]
Self_Reported_MedicalCondition = participants_withfMRI.filter(like='21062')
anxiety_code = [1]
for i in range(len(anxiety_code)):
    code = anxiety_code[i]
    print(code)
    added_item = 'Self_Reported_Treatment'
    df_fMRI[added_item] = Self_Reported_MedicalCondition['p21062'].apply(lambda row: (row == code)).replace((1,0),('True','False'))
print("number of self-reporting of medical conditions: ", df_fMRI.loc[df_fMRI[added_item]==True, added_item].shape[0])

1
number of self-reporting of medical conditions:  6673


In [13]:
df_fMRI.columns

Index(['eid', 'history_anxiety_num_criterias', 'history_anxiety_num_symptoms',
       'history_anxiety', 'ICD10', 'Diagnosed_ICD10', 'ICD9', 'Diagnosed_ICD9',
       'Self_Reported_diagnosis10', 'Self_Reported_diagnosis11',
       'Self_Reported_diagnosis12', 'Self_Reported_diagnosis13',
       'Self_Reported_diagnosis14', 'Self_Reported_diagnosis15',
       'Self_Reported_diagnosis', 'Self_Reported_MedicalCondition1615',
       'Self_Reported_MedicalCondition1287', 'Self_Reported_MedicalCondition'],
      dtype='object')

In [79]:
GAD_score_file_path = os.path.join(data_dir,'gad_score.csv')
df_fMRI[['eid','GAD7_score']].to_csv(GAD_score_file_path, index=False)

In [80]:
# subjects with anxiety
judgment_columns = ['history_anxiety', 'Diagnosed_ICD10', 'Diagnosed_ICD9', 'Self_Reported_diagnosis', 'Self_Reported_Treatment', 'Self_Reported_MedicalCondition', 'Current_Anxiety', ]
df_fMRI['anxiety'] = df_fMRI[judgment_columns].apply(lambda row: (row == True).any(), axis=1).replace((1,0),('True','False'))
print("number of anxiety: ", df_fMRI.loc[df_fMRI['anxiety']==True, 'anxiety'].shape[0])

number of anxiety:  13839


In [81]:
anxiety_fMRI_eid = df_fMRI.loc[df_fMRI['anxiety']==True, 'eid']
output_path = os.path.join(data_dir, 'needed_participants.csv')
anxiety_fMRI_eid.to_csv(output_path, index=False)

In [82]:
# get fMRI data path
imagingData_file_path = os.path.join(data_dir, 'imaging_eids.csv')
imagingInfo = pd.read_csv(imagingData_file_path)
df_data = imagingInfo.loc[imagingInfo['eid'].isin(anxiety_fMRI_eid)]

output_path = os.path.join(data_dir, 'anxiety_fMRI_participants.csv')
df_data.to_csv('anxiety_fMRI_participants.csv', index=False)

In [83]:
print(f'Number of participants with fMRI time series: {df_fMRI.shape[0]}')
for c in judgment_columns:
    n = sum(df_fMRI[c]==True)
    print(f'Number of participants with {c}: {n}')
print(f'Number of participants with anxiety symptom and fMRI time series: {anxiety_fMRI_eid.shape[0]}')
print(f'Number of participants with only history anxiety symptom and fMRI time series: {13324 - 7799}')

Number of participants with fMRI time series: 40396
Number of participants with history_anxiety: 1909
Number of participants with Diagnosed_ICD10: 1426
Number of participants with Diagnosed_ICD9: 856
Number of participants with Self_Reported_diagnosis: 2025
Number of participants with Self_Reported_Treatment: 6673
Number of participants with Self_Reported_MedicalCondition: 1096
Number of participants with Current_Anxiety: 7799
Number of participants with anxiety symptom and fMRI time series: 13839
Number of participants with only history anxiety symptom and fMRI time series: 5525


In [118]:
history_columns = ['history_anxiety', 'Diagnosed_ICD10', 'Diagnosed_ICD9', 'Self_Reported_diagnosis', 'Self_Reported_Treatment', 'Self_Reported_MedicalCondition', 'Current_Anxiety', ]
current_columns = 'Current_Anxiety'
df_fMRI['historical_anxiety'] = df_fMRI[history_columns].apply(lambda row: (row == True).any(), axis=1).replace((1,0),('True','False'))

In [124]:
#history_columns = ['history_anxiety', 'Diagnosed_ICD10', 'Diagnosed_ICD9','Current_Anxiety']
hospital_columns = ['Diagnosed_ICD10', 'Diagnosed_ICD9']
df_fMRI['hospital_anxiety'] = df_fMRI[hospital_columns].apply(lambda row: (row == True).any(), axis=1).replace((1,0),('True','False'))
sum((df_fMRI['hospital_anxiety'] == True))

2275

In [125]:
#history_columns = ['history_anxiety', 'Diagnosed_ICD10', 'Diagnosed_ICD9','Current_Anxiety']
hospital_current_columns = ['hospital_anxiety', 'Current_Anxiety']
df_fMRI['hospital_current_anxiety'] = df_fMRI[hospital_current_columns].apply(lambda row: (row == True).all(), axis=1).replace((1,0),('True','False'))
sum((df_fMRI['hospital_current_anxiety'] == True))

675

In [131]:
#history_columns = ['history_anxiety', 'Diagnosed_ICD10', 'Diagnosed_ICD9','Current_Anxiety']
hospital_not_now_columns = ['hospital_anxiety', 'Current_no_Anxiety']
df_fMRI['hospital_not_now'] = df_fMRI[hospital_not_now_columns].apply(lambda row: (row == True).all(), axis=1).replace((1,0),('True','False'))
hospital_not_now = sum((df_fMRI['hospital_not_now'] == True))
print(f'Number of participants with hospital records but not current anxiety: {hospital_not_now}')

Number of participants with hospital records but not current anxiety: 772


In [135]:
# define controls
control_fields = ['20544','20002','20514','20510','20517','20519','20511','20507','20508','20518','20513',\
                 '20506','20509','20520','20515','20516','20505','20512']
control_fields_fields_name = [fields_for_id_x(field) for field in control_fields]
control_items = ['eid']
total_items = participants_withfMRI.columns.values
for item in control_fields_fields_name:
    l = len(item)
    t = []
    for c in total_items:
        if c[:l+1] == item + '-':
            t.append(c)
            control_items.append(c)
    if len(t) == 0:
        print(item,'not exits')

p20544 not exits
p20002 not exits
p20514 not exits
p20510 not exits
p20517 not exits
p20519 not exits
p20511 not exits
p20507 not exits
p20508 not exits
p20518 not exits
p20513 not exits
p20506 not exits
p20509 not exits
p20520 not exits
p20515 not exits
p20516 not exits
p20505 not exits
p20512 not exits


In [136]:
participants_withfMRI.columns.values

array(['eid', 'p31', 'p21003_i2', 'p20016_i2', 'p6138_i2', 'p2050_i2',
       'p2060_i2', 'p2070_i2', 'p20544', 'p20436', 'p20439', 'p20440',
       'p20446', 'p20441', 'p20449', 'p20536', 'p20532', 'p20435',
       'p20450', 'p20437', 'p20421', 'p20420', 'p20538', 'p20425',
       'p20542', 'p20543', 'p20540', 'p20541', 'p20539', 'p20537',
       'p20418', 'p20426', 'p20423', 'p20429', 'p20419', 'p20422',
       'p20417', 'p20427', 'p20514', 'p20510', 'p20517', 'p20519',
       'p20511', 'p20507', 'p20508', 'p20518', 'p20513', 'p20506',
       'p20509', 'p20520', 'p20515', 'p20516', 'p20505', 'p20512',
       'p41270', 'p41271', 'p31016_i2', 'p31016_i3', 'p31018_i2',
       'p31018_i3', 'p31019_i2', 'p31019_i3', 'p31015_i2', 'p31015_i3',
       'p31014_i2', 'p31014_i3', 'p29000', 'p20002_i2_a0', 'p20002_i2_a1',
       'p20002_i2_a2', 'p20002_i2_a3', 'p20002_i2_a4', 'p20002_i2_a5',
       'p20002_i2_a6', 'p20002_i2_a7', 'p20002_i2_a8', 'p20002_i2_a9',
       'p20002_i2_a10', 'p20002_i2